In [1]:
import os
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd

In [2]:
# Конфігураційні константи
DATA_DIR = Path("../Dataset")
OUTPUT_DIR = Path("../Filtered datasets")

# мапування днів неділі в timestamp
DATE_MAP: Dict[str, str] = {
    'Monday': '2023-11-06 12:00:00',
    'Tuesday': '2023-11-07 12:00:00',
    'Wednesday': '2023-11-08 12:00:00',
    'Thursday-Morning': '2023-11-09 09:00:00',
    'Thursday-Afternoon': '2023-11-09 15:00:00',
    'Friday-Morning': '2023-11-10 09:00:00',
    'Friday-Afternoon1': '2023-11-10 13:00:00',
    'Friday-Afternoon2': '2023-11-10 17:00:00',
}

In [3]:
CATEGORY_LABELS: Dict[str, List[str]] = {
    'BENIGN': ['BENIGN'],
    'DoS': ['DDoS', 'DoS slowloris', 'DoS Hulk', 'DoS GoldenEye'],
    'PortScan': ['PortScan'],
    'Bot_Infiltration': ['Bot', 'Infiltration'],
    'Web': ['Web Attack – Brute Force', 'Web Attack – XSS', 'Web Attack – Sql Injection'],
    'FTP_SSH_Patator': ['FTP-Patator', 'SSH-Patator'],
    'Heartbleed': ['Heartbleed'],
}

GROUP_FEATURES: Dict[str, List[str]] = {
    'dos': ['Fwd Packets/s', 'Bwd Packets/s', 'Flow Duration', 'Flow IAT Min', 'Flow IAT Max', 'SYN Flag Count', 'PSH Flag Count'],
    'portscan': ['SYN Flag Count', 'FIN Flag Count', 'RST Flag Count', 'Total Fwd Packets', 'Total Backward Packets'],
    'bot_infiltration': ['Flow Duration', 'Fwd IAT Std', 'Bwd IAT Std', 'Fwd PSH Flags', 'Bwd URG Flags', 'Down/Up Ratio'],
    'web': ['Fwd Header Length', 'Bwd Header Length', 'Packet Length Variance', 'ACK Flag Count', 'Average Packet Size'],
    'ftp_ssh_patator': ['Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Bwd Avg Bytes/Bulk', 'Active Mean', 'Idle Mean'],
    'heartbleed': ['Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd IAT Min', 'Total Length of Fwd Packets', 'Packet Length Std'],
}

BASE_FEATURES: List[str] = [
    'Flow Bytes/s', 'Flow Packets/s', 'Average Packet Size', 'Down/Up Ratio',
    'Packet Length Mean', 'Packet Length Std', 'Min Packet Length', 'Max Packet Length',
    'Flow IAT Mean', 'Flow IAT Std', 'Fwd IAT Mean', 'Bwd IAT Mean',
    'SYN Flag Count', 'FIN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
    'Active Mean', 'Idle Mean', 'Subflow Fwd Packets', 'Subflow Bwd Packets',
    'Label', 'dow', 'hour', 'dow_sin', 'dow_cos', 'hour_sin', 'hour_cos'
]

STD_FEATURES: List[str] = [
    'Fwd Packet Length Std', 'Bwd Packet Length Std', 'Flow IAT Std',
    'Fwd IAT Std', 'Bwd IAT Std', 'Packet Length Std', 'Active Std', 'Idle Std'
]

In [4]:
def load_and_concat_csvs(data_dir: Path) -> pd.DataFrame:
    """
    Завантажує всі CSV-файли з директорії, додає стовпець 'Day' на основі імені файлу
    та об'єднує їх в один DataFrame.

    :param data_dir: шлях до директорії з CSV-файлами
    :return: конкатенований DataFrame з сирими даними
    """
    csv_paths = sorted(data_dir.glob("*.csv"))
    dfs: List[pd.DataFrame] = []

    for path in csv_paths:
        day_label = path.stem  # мітка дня з імені файлу
        df_temp = pd.read_csv(path)
        df_temp['Day'] = day_label
        dfs.append(df_temp)

    concatenated = pd.concat(dfs, ignore_index=True)
    concatenated.columns = concatenated.columns.str.strip()  # очищення пробілів в назвах стовпців
    return concatenated

In [5]:
def add_datetime_index(df: pd.DataFrame, date_map: Dict[str, str]) -> pd.DataFrame:
    """
    Перетворює стовпець 'Day' у datetime-індекс на основі мапи дат,
    встановлює його як індекс і видаляє стовпець 'Day'.

    :param df: DataFrame з колонкою 'Day'
    :param date_map: словник мітка -> datetime рядок
    :return: DataFrame з datetime-індексом
    """
    df['timestamp'] = pd.to_datetime(df['Day'].map(date_map))
    df = df.set_index('timestamp').drop(columns=['Day'])
    return df

In [6]:
def engineer_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Додає циклічні ознаки для дня тижня та години (синус/косинус).

    :param df: DataFrame з datetime-індексом
    :return: DataFrame з новими часовими ознаками
    """
    df['dow'] = df.index.dayofweek  # день тижня (0=Понеділок)
    df['hour'] = df.index.hour  # година доби
    df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    return df

In [7]:
def save_grouped_by_category(df: pd.DataFrame, output_dir: Path) -> None:
    """
    Для кожної категорії у CATEGORY_LABELS вибирає базові та додаткові ознаки,
    а потім зберігає результуючі CSV у вказаній директорії.

    :param df: очищений DataFrame з колонкою 'Label'
    :param output_dir: директорія для збереження CSV
    """
    output_dir.mkdir(parents=True, exist_ok=True)

    for category, labels in CATEGORY_LABELS.items():
        if category == 'BENIGN':
            features = BASE_FEATURES
        else:
            key = category.lower()
            extra = GROUP_FEATURES.get(key, [])
            features = list(dict.fromkeys(BASE_FEATURES + extra))

        subset = df[df['Label'].isin(labels)][features]
        filepath = output_dir / f"{category}.csv"
        subset.to_csv(filepath, index=False)
        print(f"Збережено {len(subset)} рядків до {filepath}")

In [8]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Очищення даних:
      - Негативні числа у числових колонках замінюються на NaN
      - Нескінченності замінюються на NaN
      - Видалення рядків з будь-якими NaN
      - Скидання індексу

    :param df: початковий або частково оброблений DataFrame
    :return: очищений DataFrame, готовий до аналізу або моделювання
    """
    # Визначення числових колонок (без деяких виключень)
    exclude = ['Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Label']
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(exclude)

    # Маска негативних значень та нескінченностей
    df[numeric_cols] = df[numeric_cols].mask(df[numeric_cols] < 0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Видалення рядків з пропусками
    df = df.dropna(axis=0, how='any')

    # Скидання індексу
    df = df.reset_index(drop=True)
    return df

In [10]:
raw_df = load_and_concat_csvs(DATA_DIR)
df_clean = clean_data(raw_df)
df_indexed = add_datetime_index(df_clean, DATE_MAP)
df_features = engineer_time_features(df_indexed)

In [12]:
def check_data_quality(df: pd.DataFrame) -> None:
    """
    Виконує перевірки даних:
      1. Наявність NaN
      2. Наявність нескінченностей
      3. Некоректні значення поля 'Destination Port'
      4. Від'ємні значення у STD_FEATURES
      5. Від'ємні значення у 'Flow Bytes/s' та 'Flow Packets/s'
      6. Від'ємні значення у всіх числових колонках (окрім винятків)
    """
    import numpy as np

    # 1) перевірка на NaN
    nan_counts = df.isna().sum()
    cols_with_nan = nan_counts[nan_counts > 0]
    print("Кількість пропусків (NaN) по колонках:")
    print(cols_with_nan)
    print("")

    # 2) перевірка на нескінченні значення
    inf_counts = df.isin([np.inf, -np.inf]).sum()
    cols_with_inf = inf_counts[inf_counts > 0]
    print("Кількість нескінченних значень по колонках:")
    print(cols_with_inf)
    print("")

    # 3) перевірка коректності порту в Destination Port
    invalid_ports = df[(df['Destination Port'] < 0) | (df['Destination Port'] > 65535)].shape[0]
    print(f"Рядків з некоректними значеннями Destination Port: {invalid_ports}")
    print("")

    # 4) перевірка від'ємних значень у стандартних відхиленнях
    std_feats = [
        "Fwd Packet Length Std", "Bwd Packet Length Std", "Flow IAT Std",
        "Fwd IAT Std", "Bwd IAT Std", "Packet Length Std", "Active Std", "Idle Std"
    ]
    neg_std_mask = (df[std_feats] < 0).any(axis=1)
    neg_std_count = df[neg_std_mask].shape[0]
    print(f"Рядків з від'ємними значеннями у STD_FEATURES: {neg_std_count}")
    print("")

    # 5) перевірка Flow Bytes/s та Flow Packets/s < 0
    flow_bytes_neg = df[df['Flow Bytes/s'] < 0].shape[0]
    flow_pkts_neg = df[df['Flow Packets/s'] < 0].shape[0]
    print(f"Flow Bytes/s < 0: {flow_bytes_neg}")
    print(f"Flow Packets/s < 0: {flow_pkts_neg}")
    print("")

    # 6) загальна перевірка від'ємних значень серед числових колонок (окрім винятків)
    exclude_cols = ['Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Label']
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(exclude_cols)
    neg_mask = df[numeric_cols] < 0
    neg_counts = neg_mask.sum()
    cols_with_neg = neg_counts[neg_counts > 0]
    print("Кількість від'ємних значень по числових колонках:")
    print(cols_with_neg)
    print("")


In [13]:
check_data_quality(raw_df)

Кількість пропусків (NaN) по колонках:
Flow Duration            115
Flow Bytes/s            2952
Flow Packets/s          2982
Flow IAT Mean            115
Flow IAT Max             115
Flow IAT Min            2891
Fwd IAT Min               17
Fwd Header Length         35
Bwd Header Length         22
Fwd Header Length.1       35
min_seg_size_forward      35
dtype: int64

Кількість нескінченних значень по колонках:
Series([], dtype: int64)

Рядків з некоректними значеннями Destination Port: 0

Рядків з від'ємними значеннями у STD_FEATURES: 0

Flow Bytes/s < 0: 0
Flow Packets/s < 0: 0

Кількість від'ємних значень по числових колонках:
Series([], dtype: int64)



In [14]:
check_data_quality(df_features)

Кількість пропусків (NaN) по колонках:
Series([], dtype: int64)

Кількість нескінченних значень по колонках:
Series([], dtype: int64)

Рядків з некоректними значеннями Destination Port: 0

Рядків з від'ємними значеннями у STD_FEATURES: 0

Flow Bytes/s < 0: 0
Flow Packets/s < 0: 0

Кількість від'ємних значень по числових колонках:
dow_cos     1850875
dow_sin      702209
hour_cos    2824951
hour_sin     799565
dtype: int64

